# Complex Ambiguity Function (CAF)

In [ ]:
%pylab inline

In [ ]:
from caf_verilog import sim_helper as sh
from caf_verilog.caf import simple_caf
from gps_helper.prn import PRN
from sk_dsp_comm import sigsys as ss
from sk_dsp_comm import digitalcom as dc

In [ ]:
prn = PRN(10)
prn2 = PRN(20)
fs = 625e3
Ns = fs / 50e3
prn_seq = prn.prn_seq()
prn_seq2 = prn2.prn_seq()
prn_seq,b = ss.NRZ_bits2(array(prn_seq), Ns)
prn_seq2,b2 = ss.NRZ_bits2(array(prn_seq2), Ns)

In [ ]:
Px,f = psd(prn_seq, 2**12, Fs=fs)
plot(f, 10*np.log10(Px))

## Single Frequency Slice

In [ ]:
n = arange(0, len(prn_seq))
theta_shift = exp(1j*2*pi*n*(50e3)/float(fs))

In [ ]:
Px, f = psd(prn_seq * theta_shift, 2**12, Fs=fs)
plot(f, 10 * np.log10(Px))

In [ ]:
Rxy, lags = dc.xcorr(prn_seq, prn_seq, len(prn_seq))
plot(lags, abs(Rxy))
ylim([0, 1])

In [ ]:
Rxy, lags = dc.xcorr(prn_seq, prn_seq * theta_shift, len(prn_seq))
plot(lags, abs(Rxy))
ylim([0, 1])

In [ ]:
f_size = 100
foas = arange(-f_size, f_size + 1) * 0.25
foas

In [ ]:
theta_shift = exp(1j*2*pi*n*(2.5)/float(fs))
prn_shift = prn_seq * theta_shift
CAF, dt = simple_caf(prn_seq * hanning(len(prn_seq)), prn_shift * hanning(len(prn_seq)), foas, fs)
CAF.shape

In [ ]:
X,Y = meshgrid(foas, dt)
contourf(X,Y,CAF)
ylabel(r'TOA/TDOA $\mu$s')
xlabel(r'FOA/FDOA (Hz)')
xlim([-5, 5])

## CAF Module

In [ ]:
from caf_verilog.caf import CAF
from caf_verilog.sim_helper import sim_shift

In [ ]:
center = 300
corr_length = 250
shift = 25
ref, rec = sim_shift(prn_seq, center, corr_length, shift=shift)
caf = CAF(ref, rec, foas)